In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.stats.multicomp as multi
from dataFilter import Replacing

df = pd.read_excel("C:/Users/Админ/Documents/GitHub/DANO_NN_2024/Data/hakaton_nn_1month.xlsx")

df["education_level_cd"] = df["education_level_cd"].replace("SCH",0)
df["education_level_cd"] = df["education_level_cd"].replace(";",0)
df["education_level_cd"] = df["education_level_cd"].replace("UGR",1)
df["education_level_cd"] = df["education_level_cd"].replace("GRD",2)
df["education_level_cd"] = df["education_level_cd"].replace("PGR",3)
df["education_level_cd"] = df["education_level_cd"].replace("ACD",4)
df["education_level_cd"] = df["education_level_cd"].replace("MGR",5)
df["education_level_cd"] = df["education_level_cd"].replace("AGR",6)
df["education_level_cd"] = df["education_level_cd"].replace("SPC",7)
df["education_level_cd"] = df["education_level_cd"].fillna(1)
df["education_level_cd"] = df["education_level_cd"].astype(int)

df["marital_status_cd"] = df["marital_status_cd"].replace(";",0)
df["marital_status_cd"] = df["marital_status_cd"].replace("MAR",2)
df["marital_status_cd"] = df["marital_status_cd"].replace("DIV",-2)
df["marital_status_cd"] = df["marital_status_cd"].replace("UNM",-1)
df["marital_status_cd"] = df["marital_status_cd"].replace("WID",-1)
df["marital_status_cd"] = df["marital_status_cd"].replace("CIV",1)
df["marital_status_cd"] = df["marital_status_cd"].replace("DLW",0)
df["marital_status_cd"] = df["marital_status_cd"].fillna(0)
df["marital_status_cd"] = df["marital_status_cd"].astype(int)

df["children_cnt"] = df["children_cnt"].fillna(0)
df["children_cnt"] = df["children_cnt"].astype(int)
df["person_monthly_income_amt"] = df["person_monthly_income_amt"].fillna(100)
df["person_monthly_income_amt"] = df["person_monthly_income_amt"].astype(int)

df["day_of_week"] = df["day_of_week"].replace("Понедельник",0)
df["day_of_week"] = df["day_of_week"].replace("Вторник",1)
df["day_of_week"] = df["day_of_week"].replace("Среда",2)
df["day_of_week"] = df["day_of_week"].replace("Четверг",3)
df["day_of_week"] = df["day_of_week"].replace("Пятница",4)
df["day_of_week"] = df["day_of_week"].replace("Суббота",5)
df["day_of_week"] = df["day_of_week"].replace("Воскресенье",6)
df["day_of_week"] = df["day_of_week"].fillna(2)

df["age"] = df["age"].fillna(25)

df["hours"] = list(map(int,[str(i).split(':')[0] for i in df["offencetime"]]))

df["public_holiday"] = df["public_holiday"].replace('t',1)
df["public_holiday"] = df["public_holiday"].replace('f',0)

C:\Users\Админ\AppData\Local\Temp\ipykernel_7484\1323637426.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["day_of_week"] = df["day_of_week"].replace("Воскресенье",6)
C:\Users\Админ\AppData\Local\Temp\ipykernel_7484\1323637426.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["public_holiday"] = df["public_holiday"].replace('f',0)


In [68]:
df["offenceprob"] = np.zeros(len(df["children_cnt"]))
for i in range(len(df["children_cnt"][:1000])):
    df["offenceprob"][i] = len(df["age"][(df["children_cnt"] == df["children_cnt"][i]) & (df["person_monthly_income_amt"] == df["person_monthly_income_amt"][i])
                                         & (df["public_holiday"] == df["public_holiday"][i]) & (df["offencetime"]==df["offencetime"][i])]) + 1

print(df["offenceprob"][:1000])

sns.histplot(df["offenceprob"][:1000])

C:\Users\Админ\AppData\Local\Temp\ipykernel_7484\787529825.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["offenceprob"][i] = len(df["age"][(df["children_cnt"] == df["children_cnt"][i]) & (df["person_monthly_income_amt"] == df["perso

In [45]:
import statsmodels.api as sm
import datetime as dt
from statsmodels.regression.linear_model import OLS

df["hours"] = list(map(int,[str(i).split(':')[0] for i in df["offencetime"]]))
df["hours"] = df["hours"][:1000].fillna(15)
df["public_holiday"] = df["public_holiday"][:1000].fillna(0)
df["children_cnt"] = df["children_cnt"][:1000]
df["children_cnt"] = df["children_cnt"].fillna(1)
df["person_monthly_income_amt"] = df["person_monthly_income_amt"][:1000].fillna(175)

print(df["children_cnt"])

X = np.array(df[["public_holiday","children_cnt","person_monthly_income_amt","hours"]])
Y = np.array(df["offenceprob"])
X = sm.add_constant(X,prepend=False)
model = OLS(Y,X)    
res = model.fit()
print(res.summary())

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
97302    1.0
97303    1.0
97304    1.0
97305    1.0
97306    1.0
Name: children_cnt, Length: 97307, dtype: float64


MissingDataError: exog contains inf or nans